# <center>Block 3: Linear programming: Dyanmic programming</center>
### <center>Alfred Galichon (NYU)</center>
## <center>`math+econ+code' masterclass on matching models, optimal transport and applications</center>
<center>© 2018-2019 by Alfred Galichon. Support from NSF grant DMS-1716489 is acknowledged. James Nesbit contributed.</center>

### Learning Objectives

* Basics of (finite-horizon, discrete) dynamic programming: Bellman's equation; forward induction, backward induction

* Markov decision processes

* Dynamic programming as linear programming: interpretation of duality

* Vectorization, Kronecker products, multidimensional arrays

### References

* Ford Jr, L. R., \& Fulkerson, D. R. (1958). Constructing maximal dynamic flows from static flows. *Operations research*, 6(3), 419-433.

* Schrijver, A. (2003). *Combinatorial optimization: polyhedra and efficiency* Vol. A. Springer. Section 12.5.c.

* Bertsekas, D. (2011), *Dynamic Programming and Optimal Control*, Vols. I and II. 3rd ed. Athena.

* Ljungqvist, Sargent (2012), *Recursive Macroeconomic Theory* 3rd ed. MIT.

* Rust (1987), Optimal replacement of GMC bus engines: an empirical model of Harold Zurcher. *Econometrica*.

## Movitation

John Rust describes the problem of Harold Zurcher, an engineer who runs a bus fleet as follows:

* each month, buses operate a stochastic number of miles

* operations costs increase with mileage (maintenance, fuel, insurance and costs of unexpected breakdowns)

* there is a fixed cost associated with overhaul (independent on mileage)

* each month, Zurcher needs to decide to send the bus to overhaul, which resets their mileage to zero, or to let them operate.

This problem is a *dynamic programming problem*. When taking the decision whether to perform the overhaul or not, Zurcher needs to compare the operation cost not only with the cost of overhaul, but also take into account the reduction in operation costs in the future periods.

While in this instance of the problem there is no externality across buses, so the buses could decide in isolation whether to go on maintenance or not, it is not hard to envision a variant of this problem where there are externalities. For instance, one may assume that there is a maximum number of buses that can go on overhaul at the same time.

We shall derive the optimal policy for Harold Zurcher, (somewhat freely) based on Rust's data.

## Linear Dynamic Programming

### Dynamic programming as linear programming

Consider a finite set of individual states $x\in\mathcal{X}$; and a set of possible actions $y\in\mathcal{Y}$; assume that at initial time, $n_{x}$ individuals in state $x$. The total number of individuals is $N=\sum_{x\in\mathcal{X}}n_{x}$. (Note that $n_{x}$ is not necessarily an integer, so it would be more correct to talk about "mass" than "number".

The immediate payoff associated with choice $y\in\mathcal{Y}$ at time $t\in\mathcal{T}=\left\{  1,...,T\right\}  $ in state $x\in\mathcal{X}$ is $u_{xy}^{t}$, discounted at time zero (typically: $u_{xy}^{t}=\beta^{t}u_{xy}$ where $\beta$ is a constant discount factor).

The individual states undergo a Markov transition. The transition depends on the $y$ chosen; hence, let $P_{x^{\prime}|xy}$ be the probability of a transition to state $x^{\prime}$ conditional on the current state being $X_{t}=x$ and the current choice being $Y_{t}=y$. For $U\in\mathbb{R}^{\mathcal{X}}$, $\left(  P^{\intercal}U\right)  _{xy}=\sum_{x^{\prime}}P_{x^{\prime}|xy}U_{x^{\prime}}$ denotes the expectation of $U_{X_{t+1}}$ given $X_{t}=x$ and $Y_{t}=y$.

Let $\pi_{xy}^{t}$ be the number of individuals who are in state $x$ and choose $y$ ("policy variable").

Define $n_{x}^{t}$ be the number of individuals in state $x$ at time $t$. We have the counting equation

\begin{align*}
\sum_{y\in\mathcal{Y}}\pi_{xy}^{t}=n_{x}^{t}.
\end{align*}

We have $n_{x}^{1}=n_{x}$ and because of the Markov transitions, 

\begin{align*}
\sum_{x\in\mathcal{X},~y\in\mathcal{Y}}P_{x^{\prime}|xy}\pi_{xy}^{t-1}=n_{x^{\prime}}^{t}~1\leq t\leq T,
\end{align*}

which express that among the individual in state $x$ who choose $y$ at time $t-1$, a fraction $P_{x^{\prime}|xy}$ transit to state $x^{\prime}$ at time $t$.

### Primal problem: Central planner's problem

The central planner's problem is:

\begin{align*}
\max_{\pi_{xy}^{t}\geq0}  &  \sum_{x\in\mathcal{X},~y\in\mathcal{Y},~t\in\mathcal{T}}\pi_{xy}^{t}u_{xy}^{t} \\
s.t.  &  \sum_{y^{\prime}\in\mathcal{Y}}\pi_{x^{\prime}y^{\prime}}^{t}=\sum_{x\in\mathcal{X},~y\in\mathcal{Y}}P_{x^{\prime}|xy}\pi_{xy}^{t-1}~\forall t\in\mathcal{T}\backslash\left\{  1\right\}  ~\left[
U_{x^{\prime}}^{t}\right] \\
&  \sum_{y^{\prime}\in\mathcal{Y}}\pi_{xy^{\prime}}^{1}=n_{x}~\left[U_{x}^{1}\right]
\end{align*}

### Dual problem

We have introduced $U_{x}^{t}$ the Lagrange multiplier associated with the constraints at time $t$. It will be convenient to also introduce $U_{x}%
^{T+1}=0$. The dual problem is

\begin{align*}
\min_{U_{x}^{t},~t\in\mathcal{T},~x\in\mathcal{X}}  &  \sum_{x\in\mathcal{X}}n_{x}U_{x}^{1} \\
s.t.~  &  U_{x}^{t}\geq u_{xy}^{t}+\sum_{x^{\prime}}U_{x^{\prime}}^{t+1}P_{x^{\prime}|xy}~\forall x\in\mathcal{X},~y\in\mathcal{Y},~t\in\mathcal{T}\backslash\left\{  T\right\} \\
&  U_{x}^{T}\geq u_{xy}^{T}~\forall x\in\mathcal{X},y\in\mathcal{Y}
\end{align*}

### Complementary slackness and Bellman's equation
By complementary slackness, we have

\begin{align*}
\pi_{xy}^{t}>0\Longrightarrow U_{x}^{t}=u_{xy}^{t}+\sum_{x^{\prime}}U_{x^{\prime}}^{t+1}P_{x^{\prime}|xy}
\end{align*}

whose interpretation is immediate: if $y$ is the optimal choice in state $x$ at time $t$, then the intertemporal payoff of $x$ at $t$ is the sum of her myopic payoff $u_{xy}^{t}$ and her expected payoff at the next step.

As a result, the dual variable is called *intertemporal payoff* in the vocable of dynamic programming. The relation yields *Bellman's equation*

<a name="bellman"></a>
\begin{align*}
U_{x}^{t}=\max_{y\in\mathcal{Y}}\left\{  u_{xy}^{t}+\sum_{x^{\prime}}U_{x^{\prime}}^{t+1}P_{x^{\prime}|xy}\right\}.
\end{align*}

## Dynamic programming as a linear program

We will need to represent matrices (such as $U_{x}^{t}$) and 3-dimensional arrays (such as $u_{xy}^{t}$). Consistent with the use in `R`, we will represent a matrix $M_{ij}$ by varying the first index first, i.e. a $2\times2$ matrix will be represented as $vec\left(M\right) = M_{11}, M_{21}, M_{12}, M_{22}$. Likewise, a $2\times2\times2$ 3-dimensional array $A$ will be represented by varying the first index first, then the second, i.e.

$vec\left(A\right) = A_{111}, A_{211}, A_{121}, A_{221}, A_{112}, A_{212}, A_{122}, A_{222}$.

In `R`, this is implemented by `c(A)` in `Matlab`, by `reshape(A,[n*m,1])`.

### Kronecker product

A very important identity is

\begin{align*}
vec\left(BXA^{\intercal}\right) = \left(  A\otimes B\right)  vec\left(X\right),
\end{align*}

where $\otimes$ is the Kronecker product: for 2x2 matrices,

\begin{align*}
A\otimes B=
\begin{pmatrix}
a_{11}B & a_{12}B\\
a_{21}B & a_{22}B
\end{pmatrix}.
\end{align*}

Recall, indices $xy\in\mathbb{R}^{\left\vert \mathcal{X}\right\vert \left\vert \mathcal{Y}\right\vert}$ are represented by varying the first index first.

Let:

* $P$ be the ($\left\vert \mathcal{X}\right\vert \left\vert \mathcal{Y}\right\vert $)$\times\left\vert \mathcal{X}\right\vert $ matrix of term $P_{x^{\prime}|xy}$.

* $J$ be the ($\left\vert \mathcal{X}\right\vert \left\vert \mathcal{Y}\right\vert $)$\times\left\vert \mathcal{X}\right\vert $ matrix of term $1\left\{  x=x^{\prime}\right\}  $. One has

\begin{align*}
J=1_{\mathcal{Y}}\otimes I_{\mathcal{X}}.
\end{align*}


* $U$ be the column vector of size $\left\vert \mathcal{X}\right\vert \left\vert T\right\vert $ obtained by stacking the vectors $U^{1}$,...,$U^{T}$.

* $b$ be the column vector of size $\left\vert \mathcal{X}\right\vert \left\vert T\right\vert $ whose $\left\vert \mathcal{X}\right\vert $ first terms are the terms of $n$, and whose other terms are zero.

* $u$ be the column vector of size $\left\vert \mathcal{X}\right\vert \left\vert \mathcal{Y}\right\vert \left\vert T\right\vert $ obtained by stacking the vectors $u^{1}$,..., $u^{T}$.

* $\pi$ be the vector obtained by stacking the vectors $\pi^{1}$,...,$\pi^{T}$.



$A$ is the $\left\vert T\right\vert \left\vert \mathcal{X}\right\vert \left\vert \mathcal{Y}\right\vert \times\left\vert T\right\vert \left\vert
\mathcal{X}\right\vert $ matrix

\begin{align*}
A=
\begin{pmatrix}
J & -P & 0 & \cdots & 0\\
0 & J & \ddots & \ddots & \vdots\\
\vdots & \ddots & \ddots & -P & 0\\
\vdots &  & \ddots & J & -P\\
0 & \cdots & \cdots & 0 & J
\end{pmatrix}
\end{align*}

Letting $N_{\mathcal{T}}$ be the $T\times T$ matrix given by

\begin{align*}
N_{\mathcal{T}}=
\begin{pmatrix}
0 & 1 & 0 & \cdots & 0\\
\vdots & \ddots & \ddots &  & \vdots\\
&  & \ddots & \ddots & 0\\
\vdots &  &  & \ddots & 1\\
0 & \cdots &  & \cdots & 0
\end{pmatrix}
\end{align*}

the constraint matrix can be reexpressed as

\begin{align*}
A=I_{\mathcal{T}}\otimes J-N_{\mathcal{T}}\otimes P=I_{\mathcal{T}}\otimes1_{\mathcal{Y}}\otimes I_{\mathcal{X}}-N_{\mathcal{T}}\otimes P.
\end{align*}

Although we'll see much faster direct methods, the primal and dual problems could be solved by a black-box linear programming solver.

Then the primal problem expresses as

\begin{align*}
\max_{\pi\geq0}  &  \, u^{\intercal}\pi\\
s.t.~  &  A^{\intercal}\pi=b~\left[U\right]
\end{align*}

while the dual problem is given by

\begin{align*}
\min_{U}  & \, b^{\intercal}U\\
s.t.~  &  AU\geq u~\left[\pi\right]  .
\end{align*}

But there is in fact a much faster way to compute the primal and dual solutions without having to use the full power of a linear programming solver. Along with the fact that $U^{T+1}=0$, [Bellman's equation](#bellman) implies that there is a particularly simple method to obtain the dual variables $U^{t}$, by solving recursively backward in time, from $t=T$ to $t=1$. This method is called *backward induction*:

---
**Algorithm**
Set $U^{T+1}=0$

For $t=T$ down to $1$, set $U_{x}^{t}:=\max_{y\in\mathcal{Y}}\left\{u_{xy}^{t}+\sum_{x^{\prime}}U_{x^{\prime}}^{t+1}P_{x^{\prime}|xy}\right\}$.

---

The primal variables $\pi^{t}$ are then deduced also by recursion, but this time forward in time from $t=1$ to $t=T-1$, by the so-called *forward induction* method:

---
**Algorithm**
1. Set $n^{1}=n$ and compute $\left(  U^{t}\right)$ by backward induction.

2. For {$t=1$ to }$T$, pick $\pi^{t}$ such that $\pi_{xy}^{t}/n_{x}^{t}$ is a probability measure supported in the set

\begin{align*}
\left\{  y:U_{x}^{t}=u_{xy}^{t}+\sum_{x^{\prime}}U_{x^{\prime}}^{t+1}P_{x^{\prime}|xy}\right\}  .
\end{align*}


3. Set $n_{x^{\prime}}^{t+1}:=\sum_{x\in\mathcal{X},~y\in\mathcal{Y}}P_{x^{\prime}|xy}\pi_{xy}^{t-1}$

End

---

### Remarks 

1. The dual variable is $U$ always unique (this follows from the backward induction computation); the primal variable is not, as there may be ties between several states.

2. The computation by the combination of the backward and forward algorithms is much faster than the computation by a black-box linear programming solver.

3. However, as soon as we introduce capacity constraints, the computation by backward induction no longer works, and the linear programming formulation is useful.

# Back to Harold Zurcker

The state $x\in\mathcal{X}=\left\{x_{0},...,\bar{x}\right\}$ of each bus at each period $t$ is the mileage since last overhaul. The transition between states is as follows:
* When no overhaul is performed, these states undergo random transitions (depending on how much the bus is used): $x_{i}\rightarrow x_{i^{\prime}}$ with some probability $P_{i^{\prime}|i}$, where $i^{\prime}\geq i$.

* When overhaul is performed on a bus, the state is restored to the zero state $x_{0}$.

There is a fixed cost $C$ associated with overhaul (independent of mileage), while operations costs $c\left(  x\right)$ increase with mileage (maintenance, fuel, insurance and costs of nexpected breakdowns).

### Implementation

Assume the states are discretized into $12,500$ mile brackets. 

There are $30$ states, so $\mathcal{X}=\left\{  1,...,30\right\}$: `nbX = 30`.

The choice set is $\mathcal{Y}=\left\{  y_{0}=1,y_{1}=2\right\}$ (operate or overhaul): `nbY = 2`.

There are $40$ periods (quarter years over $10$ years): `nbT = 40`.

In [105]:
library("Matrix")
library("gurobi")
nbX = 3  #30
nbT = 2  #40
nbY = 2  # choice set: 1=run as usual; 2=overhaul

#### Transitions:
* If no overhaul is performed, each state but the last one has a probability $25\%$ of transiting to the next one, and probability $75\%$ of remaining the same. The last state transits to $1$ with probability $25\%$ (overhaul is performed when beyond last state).

* If overhaul is performed, the state transits to $1$ for sure.

We are going to solve the dual problem 
\begin{align*}
\min_{U}  & \, b^{\intercal}U\\
s.t.~  &  AU\geq u~\left[  \pi\right] 
\end{align*}

First let's construct our constraint matrix $A$. We build the transition matrix $P_{x^{\prime}|xy}$ matrix of dimension `nbXnbY`$\times$ `nbX`

Let
\begin{align*}
L_{\mathcal{X}}=%
\begin{pmatrix}
0 & 1 & 0 & 0\\
0 & \ddots & \ddots & 0\\
0 & \ddots & \ddots & 1\\
1 & 0 & 0 & 0
\end{pmatrix}
\text{ and }R_{\mathcal{X}}=%
\begin{pmatrix}
1 & 0 & \cdots & 0\\
1 & \vdots & \ddots  & \vdots\\
1 & \vdots &  \ddots & \vdots\\
1 & 0 & \cdots & 0
\end{pmatrix}
\end{align*}
so that $P$ is given by
\begin{align*}
P=1_{y_{0}}\otimes\left(  0.75I_{\mathcal{X}}+0.25L_{\mathcal{X}}\right)
+1_{y_{1}}\otimes R_{\mathcal{X}}%
\end{align*}

Which in R looks like

In [106]:
IdX = Diagonal(nbX)
LX = sparseMatrix(c(nbX, 1:(nbX - 1)), 1:nbX)
RX = sparseMatrix(1:nbX, rep(1, nbX), dims = c(nbX, nbX))
P = kronecker(c(1, 0), 0.75 * IdX + 0.25 * LX) + kronecker(c(0, 1), RX)

* Let's make sure that we encoded rightly matrix of Markov transitions $P$:

In [107]:
colnames(P) = paste0("x",1:nbX)
rownames(P) = outer(paste0("x",1:nbX,","),paste0("y",1:nbY),FUN=paste0)
P

6 x 3 sparse Matrix of class "dgTMatrix"
        x1   x2   x3
x1,y1 0.75 0.25 .   
x2,y1 .    0.75 0.25
x3,y1 0.25 .    0.75
x1,y2 1.00 .    .   
x2,y2 1.00 .    .   
x3,y2 1.00 .    .   

* Looks about right!

Recall the constraint matrix $A$ can be expressed as
\begin{align*}
A &= I_{\mathcal{T}}\otimes J-N_{\mathcal{T}}\otimes P \\
  &= I_{\mathcal{T}} \otimes1_{\mathcal{Y}}\otimes I_{\mathcal{X}}-N_{\mathcal{T}}\otimes P.
\end{align*}
where
\begin{align*}
N_{\mathcal{T}}=
\begin{pmatrix}
0 & 1 & 0 & \cdots & 0\\
\vdots & \ddots & \ddots &  & \vdots\\
&  & \ddots & \ddots & 0\\
\vdots &  &  & \ddots & 1\\
0 & \cdots &  & \cdots & 0
\end{pmatrix}
\end{align*}

In [108]:
IdT = Diagonal(nbT)
NT = sparseMatrix(1:(nbT - 1), 2:nbT, dims = c(nbT, nbT))
A = kronecker(kronecker(IdT, matrix(1, nbY, 1)), IdX) - kronecker(NT, P)

* Time to take a look at matrix A

In [109]:
rownames(A) = c(outer(paste0("t",1:nbT,","),rownames(P),FUN=paste0))
colnames(A) = c(outer(paste0("t",1:nbT,","),colnames(P),FUN=paste0))
A

12 x 6 sparse Matrix of class "dgCMatrix"
         t1,x1 t2,x1 t1,x2 t2,x2 t1,x3 t2,x3
t1,x1,y1     1     .     . -0.75 -0.25  .   
t2,x1,y1     .     1     .  .    -0.75 -0.25
t1,x2,y1     .     .     1 -0.25  .    -0.75
t2,x2,y1     1     .     . -1.00  .     .   
t1,x3,y1     .     1     . -1.00  .     .   
t2,x3,y1     .     .     1 -1.00  .     .   
t1,x1,y2     .     .     .  1.00  .     .   
t2,x1,y2     .     .     .  .     1.00  .   
t1,x2,y2     .     .     .  .     .     1.00
t2,x2,y2     .     .     .  1.00  .     .   
t1,x3,y2     .     .     .  .     1.00  .   
t2,x3,y2     .     .     .  .     .     1.00

#### Costs:
* The cost of replacing an engine is $C=\$8,000$ (in $1985$ dollars).

* The operations cost is $c\left(  x\right)  =x\times5.10^{2}.$

The discount factor is $\beta=0.9$.

In [110]:
overhaulCost = 8000
maintCost = function(x) (x * 500)
beta = 0.9

Next, we build $u_{xyt}$ 
* First the $u_{xy}$'s so that $u_{x1}=-x\times5.10^{2}$ for $x<\bar{x}$, and $u_{\bar{x}1}=-C$, while $u_{x2}=-C$ for all $x$.

* Next the $u_{xyt}$ so that $u_{xyt}=u_{xy}\beta^{t}=vec\left(\left(\beta^{1},...,\beta^{T}\right)  \otimes u_{xy}\right)$

Finially we build $b_{xt}$

In [111]:
n1_x = rep(1, nbX)
u_xy = c(-maintCost(1:(nbX - 1)), rep(-overhaulCost, nbX + 1))
u_xyt = c(kronecker(beta^(1:nbT), u_xy))

b_xt = c(n1_x, rep(0, nbX * (nbT - 1)))

result = gurobi(list(A = A, obj = c(b_xt), modelsense = "min", rhs = u_xyt, sense = ">", 
    lb = -Inf), params = list(OutputFlag = 0))

U_x_t_gurobi = array(result$x, dim = c(nbX, nbT))
pi_x_y_t = array(result$pi, dim = c(nbX, nbY, nbT))

print(U_x_t_gurobi[, 1])

[1]  -956.25 -3127.50 -7605.00


## Backward induction

The smarter way to approach this problem is of course using backwards induction

In [112]:
U_x_t = matrix(0, nbX, nbT)
contVals = apply(X = array(u_xyt, dim = c(nbX, nbY, nbT))[, , nbT], FUN = max, MARGIN = 1)
U_x_t[, nbT] = contVals

for (t in (nbT - 1):1) {
    myopic = array(u_xyt, dim = c(nbX, nbY, nbT))[, , t]
    Econtvals = matrix(P %*% contVals, nrow = nbX)
    
    contVals = apply(X = myopic + Econtvals, FUN = max, MARGIN = 1)
    U_x_t[, t] = contVals
}

Which give identical solutions to the ones obtained when using linear programming:

In [113]:
print(U_x_t_gurobi[, 1] - U_x_t[, 1])

[1] 0 0 0



## Capacity constraints
Now assume that the total number of alternatives $y$ chosen at time $t$ cannot be more than $m_{y}^{t}$ (either because the workshop has a maximal capacity, or because operations require a minimum number of buses in service).

The primal problem becomes

\begin{align*}
\max_{\pi_{xy}^{t}\geq0}  &  \sum_{x\in\mathcal{X},~y\in\mathcal{Y}%
,~t\in\mathcal{T}}\pi_{xy}^{t}u_{xy}^{t}\\
s.t.  &  \sum_{y^{\prime}\in\mathcal{Y}}\pi_{x^{\prime}y^{\prime}}^{t}%
=\sum_{x\in\mathcal{X},~y\in\mathcal{Y}}P_{x^{\prime}|xy}\pi_{xy}%
^{t-1}~\left[  U_{x^{\prime}}^{t}\right] \nonumber\\
&  \sum_{y^{\prime}\in\mathcal{Y}}\pi_{xy^{\prime}}^{1}=n_{x}~\left[
U_{x}^{1}\right] \nonumber\\
&  \sum_{x\in\mathcal{X}}\pi_{xy}^{t}\leq m_{y}^{t}~[\lambda_{y}^{t}]\nonumber
\end{align*}

Let us describe this problem in matrix form. Let $\tilde{\pi}^{t}$ be the matrix of term $\pi_{xy}^{t}$ for fixed $t$. The last constraint rewrites $1_{\mathcal{X}}^{\intercal}\tilde{\pi}^{t}\leq\left(  m^{t}\right) ^{\intercal}$. Vectorizing yields $vec\left(  1_{\mathcal{X}}^{\intercal}\tilde{\pi}^{t}I_{\mathcal{Y}}\right)  \leq vec\left(  m^{t}\right)  $, thus

\begin{align*}
\left(  I_{\mathcal{Y}}\otimes1_{\mathcal{X}}^{\intercal}\right)  vec\left(
\tilde{\pi}^{t}\right)  \leq vec\left(  m^{t}\right)  ,
\end{align*}

hence the constraint rewrites $B^{\intercal}\pi\leq m$, with

\begin{align*}
B=
\begin{pmatrix}
I_{\mathcal{Y}}\otimes1_{\mathcal{X}} & 0 & \cdots & 0\\
0 & \ddots & \ddots & \vdots\\
\vdots & \ddots & \ddots & 0\\
0 & \cdots & 0 & I_{\mathcal{Y}}\otimes1_{\mathcal{X}}%
\end{pmatrix}
=I_{\mathcal{T}}\otimes I_{\mathcal{Y}}\otimes1_{\mathcal{X}}.
\end{align*}

The primal problem then writes

\begin{align*}
\max_{\pi\geq0}  &  u^{\intercal}\pi\\
s.t.~  &  A^{\intercal}\pi=b~\left[  U\right] \\
&  B^{\intercal}\pi\leq m~\left[  \Lambda\right]
\end{align*}

whose dual is

\begin{align*}
\min_{U,\Lambda\geq0}  &  b^{\intercal}U+m^{\intercal}\Lambda\\
s.t.~  &  AU+B\Lambda\geq u~\left[  \pi\right]
\end{align*}

The dual becomes
\begin{align*}
\min_{U_{x}^{t},\lambda_{y}^{t}\geq0}  &  \sum_{x\in\mathcal{X}}n_{x}U_{x}%
^{1}+\sum_{x\in\mathcal{X}}\sum_{t\in\mathcal{T}}m_{y}\lambda_{y}^{t}\\
s.t.~  &  U_{x}^{t}\geq u_{xy}^{t}-\lambda_{y}^{t}+\sum_{x^{\prime}%
}U_{x^{\prime}}^{t+1}P_{x^{\prime}|xy}~\forall x\in\mathcal{X},~y\in
\mathcal{Y},~t\in\mathcal{T}\backslash\left\{  T\right\} \nonumber\\
&  U_{x}^{T}\geq u_{xy}^{T}~\forall x\in\mathcal{X},y\in\mathcal{Y}\nonumber
\end{align*}

and $\lambda_{y}^{t}$ interprets as the shadow price of alternative $y$ at
time $t$.

This constraint is extremely easy to code.

In [114]:
m_y_t = rep(c(sum(n1_x), 1), nbT)
B = kronecker(kronecker(IdT, sparseMatrix(1:nbY, 1:nbY)), matrix(1, nbX, 1))

result = gurobi(list(A = cbind(A, B), obj = c(b_xt, m_y_t), modelsense = "min", rhs = u_xyt, 
    sense = ">", lb = c(rep(-Inf, nbX * nbT), rep(0, nbY * nbT))), params = list(OutputFlag = 0))

U_x_t_gurobi = array(result$x, dim = c(nbX, nbT))

print(U_x_t_gurobi[, 1])

[1]   -956.25  -3127.50 -12161.25


In this case, the linear programming solver is our only way to compute the solution. The backward induction algorithm does not work because of the capacity constraint.

## Infinite-horizon case

* Assume $u_{xy}^{t}=\beta^{t}u_{xy}$, and look for $U_{x}^{t}$ under the
form $U_{x}^{t}=\beta^{t}U_{x}$. A solution $\left(  \pi,U\right)  $ to the
stationary equilibrium is such that:
\begin{align*}
\left\{
\begin{array}
[c]{l}%
U_{x}\geq u_{xy}+\beta\sum_{x^{\prime}}U_{x^{\prime}}P_{x^{\prime}|xy}\\
\sum_{y^{\prime}\in\mathcal{Y}}\pi_{x^{\prime}y^{\prime}}=\sum_{x\in
\mathcal{X},~y\in\mathcal{Y}}P_{x^{\prime}|xy}\pi_{xy}\\
\pi_{xy}^{t}>0\Longrightarrow U_{x}=u_{xy}+\beta\sum_{x^{\prime}}U_{x^{\prime
}}P_{x^{\prime}|xy}%
\end{array}
\right.
\end{align*}

* Under a vectorized form, this is
\begin{align*}
\left\{
\begin{array}
[c]{l}%
\pi\geq0\\
\left(  1_{\mathcal{Y}}\otimes I_{\mathcal{X}}-\beta P\right)  U\geq u\\
\pi^{\intercal}\left(  1_{\mathcal{Y}}\otimes I_{\mathcal{X}}-P\right)  =0\\
\pi^{\intercal}\left(  1_{\mathcal{Y}}\otimes I_{\mathcal{X}}-\beta P\right)
U=\pi^{\intercal}u
\end{array}
\right.
\end{align*}

* Note that unless $β=1$, these conditions are not the complementary slackness conditions associated with a linear programming problem. In fact, they are what is called a nonlinear complementarity problem (NCP).



##    Stationary solution, unit discount rate

* When $β=1$, one may interpret these at the complementary slackness conditions associated with a linear programming problem, namely
\begin{align*}
\max_{\pi\geq0}  &  \pi^{\intercal}u\\
s.t.~  &  \left(  1_{\mathcal{Y}}\otimes I_{\mathcal{X}%
}-P\right)^{\intercal} \pi  =0
\end{align*}
whose dual is
\begin{align*}
\min_{U}  &  0^{\intercal}U\\
s.t.~  &  \left(  1_{\mathcal{Y}}\otimes I_{\mathcal{X}}-P\right)  U\geq u
\end{align*}

* This can be immediately coded in Gurobi as:

In [120]:
stationaryResult = gurobi(list(A = kronecker(matrix(1:nbY,nbY,1),Diagonal(nbX)) , obj = rep(0,nbX), modelsense = "min", rhs = u_xy, sense = ">", 
    lb = -Inf))
U_snd  = stationaryResult$x - stationaryResult$x[1] 
pi_snd = stationaryResult$pi

Optimize a model with 6 rows, 3 columns and 6 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+02, 8e+03]
Presolve removed 6 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  0.000000000e+00


In [123]:
u_xy

[1]  -500 -1000 -8000 -8000 -8000 -8000